# 📖 [LMX Tutorial] Absolute and Relative motion (in Python)

Based on the following WMX API document:

http://download.movensys.com:8222/doc/wmx3/en/html/api/page_WMXDOC_TUTORIAL_SEC3_1_ABS_REL.html

### ⭐ Start LMX engine manually

Note that this is required only for LMX Python programs.

In [ ]:
!/opt/lmx/bin/lmx-start-engine

## Import WMX library and Initialize a WMX3 API device

In [ ]:
# Import WMX3 API library
from WMX3ApiPython import *
# Import WMX3 Utils library for Python Notebook
from WMX3UtilPython import *

# When all the devices are done, the WMX3 engine will also terminate.	
wmx3_api = WMX3Api()
wmx3_core_motion = CoreMotion(wmx3_api)

# Create a device.
wmx3_api.CreateDevice('/opt/lmx', DeviceType.DeviceTypeNormal, INFINITE)

# Set wmx3_api Name.
wmx3_api.SetDeviceName('device')

# Initialize WMX3 Python Utility
wmx3_pyutil = WMX3PyUtil(wmx3_api)

# Settings for Jupyter notebooks
%matplotlib inline
%config InlineBackend.figure_format='retina'

## Set the device name and start communication

In [ ]:
# Set wmx3_api Name.
ret = wmx3_api.SetDeviceName('MotorControl')
if ret != ErrorCode.PyNone:
    check_errorcode("SetDeviceName", ret)
    
# Start Communication.
ret = wmx3_api.StartCommunication(INFINITE)
if ret != ErrorCode.PyNone:
    check_errorcode("StartCommunication", ret)

print(f'StartCommunication succeeded')

In [ ]:
# Get created device state.
ret, devInfo = wmx3_api.GetAllDevices()
if ret != ErrorCode.PyNone:
    check_errorcode("GetAllDevices", ret)
    
# Display the acquired device.
print(f'Device Id: {devInfo.GetDevices(0).id}, Name: {devInfo.GetDevices(0).name}')

## Set servo ON ▶️

In [ ]:
wmx3_core_motion.axisControl.SetServoOn(0, 1)
while True:
    # wmx3_core_motion.GetStatus(coremotion_status)
    ret, coremotion_status = wmx3_core_motion.GetStatus()
    if coremotion_status.GetAxesStatus(0).servoOn:
        break
    sleep(0.1)

## 1. Absolute position command (`StartPos`)

Refer to the following API documentation:

http://download.movensys.com:8222/doc/wmx3/en/html/api/classwmx3_api_1_1_motion.html#_CPPv4N7wmx3Api6Motion8StartPosEjP17TriggerPosCommand

In [ ]:
# Create a command value.
pos_command = Motion_PosCommand()

# Set position command parameters
pos_command.axis = 0
pos_command.profile.type = ProfileType.Trapezoidal
pos_command.profile.velocity = 2000
pos_command.profile.acc = 500
pos_command.profile.dec = 500

# Execute absolute position command to 10000
pos_command.target = 10000

log_channel = wmx3_pyutil.start_log()
if (log_channel < 0):
    raise RuntimeError("Failed to execute startLog")

# Rotate the motor at the specified speed.
err = wmx3_core_motion.motion.StartPos(pos_command)

if err != ErrorCode.PyNone:
    error_string = wmx3_core_motion.ErrorToString(err)
    print("Failed to execute motion. Error=%d (%s)\n", err, error_string)

# Wait for the motor to stop
err = wmx3_core_motion.motion.Wait(0)

wmx3_pyutil.pause_log(log_channel)

### 1.1 Draw the position and velocity plots for absolute motion 📈

In [ ]:
wmx3_pyutil.draw_plots('Absolute position command (StartPos)')

## 2. Relative position command (`StartMov`)

Refer to the following API documentation:

http://download.movensys.com:8222/doc/wmx3/en/html/api/classwmx3_api_1_1_motion.html#_CPPv4N7wmx3Api6Motion8StartMovEP10PosCommand

In [ ]:
# Execute relative position command of 10000
pos_command.target = 1000
pos_command.profile.velocity = 200
pos_command.profile.acc = 50
pos_command.profile.dec = 50

log_channel = wmx3_pyutil.start_log()
if (log_channel < 0):
    raise RuntimeError("Failed to execute startLog")

err = wmx3_core_motion.motion.StartMov(pos_command)

if err != ErrorCode.PyNone:
    error_string = wmx3_core_motion.ErrorToString(err)
    print("Failed to execute motion. Error=%d (%s)\n", err, error_string)

# Wait for the motor to stop
err = wmx3_core_motion.motion.Wait(0)

wmx3_pyutil.pause_log(log_channel)

### 1.1 Draw the position and velocity plots for relative motion 📈

In [ ]:
wmx3_pyutil.draw_plots('Relative position command (StartMov)')

## Set servo OFF ⏹️

In [ ]:
wmx3_core_motion.axisControl.SetServoOn(0, 0)
while True:
    # wmx3_core_motion.GetStatus(coremotion_status)
    ret, coremotion_status = wmx3_core_motion.GetStatus()
    if coremotion_status.GetAxesStatus(0).servoOn:
        break
    sleep(0.1)

## Stop communication and close the WMX3 API device

In [ ]:
# ----------------------
# Stop Communication.
# ----------------------
wmx3_api.StopCommunication(INFINITE)

# Close wmx3_api.
wmx3_api.CloseDevice()

## (Optional) Stop LMX engine

In [ ]:
!/opt/lmx/bin/lmx-stop-engine